# Decode context from spikes or facemap

1 - either use all annotated & uploaded ephys sessions as input or provide a list of session_ids

2 - set a savepath and filename for the output - one .pkl file per session

3 - set parameters - descriptions below

4 - run decoding!

In [1]:
import sys
# sys.path.append(r"C:\Users\shailaja.akella\Dropbox (Personal)\DR\dynamic_routing_analysis_ethan\src")

import npc_lims
from dynamic_routing_analysis import decoding_utils
from npc_sessions import DynamicRoutingSession
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
#1A get all uploaded & annotated ephys sessions

ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, 
                                                            is_annotated=True, project='DynamicRouting', issues = []))

In [ ]:
#1B alternatively, provide a list of session ids:
session_id_list=['712815_2024-05-22','708016_2024-05-01','664851_2023-11-14','702136_2024-03-05','686176_2023-12-05']
session_list=[]
for ss in session_id_list:
    session_list.append(npc_lims.get_session_info(ss))
ephys_sessions=tuple(session_list)
ephys_sessions

In [ ]:
#2 set savepath and filename
savepath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\test"
filename='decoding_results_test'

except_list={}

#3 set parameters
#linear shift decoding currently just takes the average firing rate over all bins defined here
spikes_binsize=0.2 #bin size in seconds
spikes_time_before=0.2 #time before the stimulus per trial
spikes_time_after=0.01 #time after the stimulus per trial

# #not used for linear shift decoding, were used in a previous iteration of decoding analysis
# decoder_binsize=0.2
# decoder_time_before=0.2
# decoder_time_after=0.1


params = {
    'n_units': ['all'], #number of units to sample for each area (list)
    'n_repeats': 25,  # number of times to repeat decoding with different randomly sampled units
    'input_data_type': 'spikes',  # spikes or facemap or LP
    'vid_angle_facemotion': 'face', # behavior, face, eye
    'vid_angle_LP': 'behavior',
    'central_section': '4_blocks_plus',
    # for linear shift decoding, how many trials to use for the shift. '4_blocks_plus' is best
    'exclude_cue_trials': False,  # option to totally exclude autorewarded trials
    'n_unit_threshold': 20,  # minimum number of units to include an area in the analysis
    'keep_n_SVDs': 500,  # number of SVD components to keep for facemap data
    'LP_parts_to_keep': ['ear_base_l', 'eye_bottom_l', 'jaw', 'nose_tip', 'whisker_pad_l_side'],
    'spikes_binsize': spikes_binsize,
    'spikes_time_before': spikes_time_before,
    'spikes_time_after': spikes_time_after,
    # 'decoder_binsize':decoder_binsize,
    # 'decoder_time_before':decoder_time_before,
    # 'decoder_time_after':decoder_time_after,
    'savepath': savepath,
    'filename': filename,
    'use_structure_probe': True,  # if True, appedn probe name to area name when multiple probes in the same area
    'crossval': '5_fold',  # '5_fold' or 'blockwise' - blockwise untested with linear shift
    'labels_as_index': True,  # convert labels (context names) to index [0,1]
    'decoder_type': 'linearSVC',  # 'linearSVC' or 'LDA' or 'RandomForest' or 'LogisticRegression'
    'only_use_all_units': False, #if True, do not run decoding with different areas, only with all areas -- for debugging
}


for ephys_session in ephys_sessions:
    if os.path.exists(savepath + '/' + ephys_session.id[:17] + '_' + filename + '.pkl'): 
        print(ephys_session.id[:17] + ' completed, skipping...')    
        continue
    try:
        session = DynamicRoutingSession(ephys_session.id)
        print(session.id+' loaded')
        if 'structure' in session.electrodes[:].columns:
            session_info=ephys_session
            session_id=str(session_info.id)
            trials=pd.read_parquet(
                npc_lims.get_cache_path('trials',session_id,'any')
            )
            units=pd.read_parquet(
                npc_lims.get_cache_path('units',session_id,'any')
            )
            decoding_utils.decode_context_with_linear_shift(session=None,params=params,trials=trials,units=units,session_info=session_info)

            #find path of decoder result
            file_path= [os.path.join(savepath, ephys_session.id[:17] + '_' + filename + '.pkl')]

            decoding_results=decoding_utils.concat_decoder_results(file_path,savepath=savepath,return_table=True,single_session=True)

            #find n_units to loop through for next step
            n_units=[]
            for col in decoding_results.filter(like='true_accuracy_').columns.values:
                if len(col.split('_'))==3:
                    temp_n_units=col.split('_')[2]
                    try:
                        n_units.append(int(temp_n_units))
                    except:
                        n_units.append(temp_n_units)
                else:
                    n_units.append(None)

            for nu in n_units:
                decoding_utils.concat_trialwise_decoder_results(file_path,savepath=savepath,return_table=False,n_units=nu,single_session=True)

        else:
            print('no structure column found in electrodes table, moving to next recording')
        session=[]
    except Exception as e:
        except_list[session.id]=repr(e)


In [5]:
# savepath=r"\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\test"
# filename='decoding_results_test'

decoding_utils.concat_decoder_summary_tables(savepath)

In [ ]:
except_list